In [ ]:
#webcrawling
# 라이브러리 
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time
from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool #pip install pathos
import re
import pymysql

# 웹페이지 스크롤을 여러번 하기때문에 함수로 만듬
def scoll(driver):
        SCROLL_PAUSE_TIME = 1
        last_height = driver.execute_script("return document.body.scrollHeight")        
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)                                                
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:                                                
                break
            last_height = new_height
            
# 웹브라우져는 기본적으로 싱글스레드이기 때문에 오류가 생겨 멀티스레드로 하기위한 클래스
class Parser():
    #생성자
    def __init__(self):
        self.pool = Pool(processes=3)
    
    #내부메소드
    def open_browser(self, site):
        driver = webdriver.Chrome('./chromedriver.exe')
        driver.get(site)
        scoll(driver)
        return driver.page_source

Free_Cancell=False

# 웹페이지 수만큼 반복
for i in range(3):
    i+=1
    url="https://www.myrealtrip.com/themes/1088/offers?order=recommended&page={}&per=32".format(i)
    headers={"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    
    #웹브라우져 생성
    driver = webdriver.Chrome()
    driver.get(url);
    
    #스크롤하여 데이터추가로드
    scoll(driver)
    
    #최신버전의 경우 first_elems = driver.find_elements(By.CLASS_NAME,"OfferHorizontalCard-module__container--XWrxS.CardShare-module__container--lu1iP")
    first_elems = driver.find_elements_by_class_name("OfferHorizontalCard-module__container--XWrxS.CardShare-module__container--lu1iP")
       
    #웹페이지에서 원하는 엘리먼트 수 만큼 반복
    for elem in first_elems:
        
        #멀티스레드 생성
        parser = Parser()
        
        #무료취소가 있는지 없는지 확인
        try:
            elem_temp = elem.find_element_by_class_name("CardFreeRefundSticker-module__container--W1Wtx")
            Free_Cancell = True 
        except:
            Free_Cancell = False 
        
        #새로운사이트 접속
        soup = BeautifulSoup(parser.open_browser(elem.get_attribute('href')), "lxml")
        
        #필요한 데이터 추출
        name_temp=soup.find('h1', {'class':"OfferHeaderTitle-module__container--ZcKff"})
        name_filter = re.compile(r'[^ -=+,#/?:^.*~ㆍ!A-Za-z0-9가-힣+]') # 필터
        name=name_filter.sub('', name_temp.get_text())#필터를 이용해 리플레이스
        star_jum=soup.find('span', {'class':"OfferHeaderReview-module__score--PFNsE"})
        review_num=soup.find('span', {'class':"OfferHeaderReview-module__count--VmY1D"})
        price=soup.find('strong', {'class':"OfferPrice-module__price--rSHhS"})
        Wish_list=soup.find('p', {'class':"OfferSideBarMainWishlist-module__desc--W5of9"})
        
        #동일한 클래스를 가지고 있기때문에 분리구분
        country_city=soup.find_all('span', {'class':"OfferHeaderLocation-module__name--PhOlA"})
        country=country_city[0].get_text()
        city=country_city[1].get_text()
        
        #db연결
        db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
        ecommerce = db.cursor()
        
        #sql문 작성
        sql = """INSERT INTO trip VALUES("{0}", "{1}", '''{2}''', {3}, {4}, {5}, {6},{7});""".format(country, city,name,re.sub(r'[^0-9]', '', price.get_text()), star_jum.get_text(),re.sub(r'[^0-9]', '', review_num.get_text()),re.sub(r'[^0-9]', '', Wish_list.get_text()),Free_Cancell)
        ecommerce.execute(sql)
        
        #db에 전송 및 연결해제
        db.commit()
        db.close()
    
    #웹브라우져 해제
    driver.close()
    driver.quit()   
    

In [ ]:
#테이블 생성
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
ecommerce = db.cursor()
sql = """
    create table trip (
        Country  varchar(30) not null,
        City  varchar(30),
        Package varchar(400),
        Price int,
        Rate DECIMAL(2,1),
        Review int,
        Wish int,
        Free_Cancell BOOLEAN 
    );
"""
ecommerce.execute(sql)
db.commit()
db.close()